Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [5]:
import json 
import pandas as pd
import os
import csv

**Weather** **Preprocessing**

In [16]:
def weather_comp(data):
  column_names = ['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM' , 'visibility' , 'pressure' , 'cloudcover' ,'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'humidity','time','date','airport']
  df = pd.DataFrame(columns = column_names)
  w = list(data.values())
  first = w[0] #this has {'weather':[{day1},{day2},{day....}]}
  weather = first['weather'] #this is a list, with a dictionaries, each dict with each day
  for k in range(len(weather)):
    weather1 = weather[k] #this is a dict with stuff of each day
    l = len(list(weather1['hourly']))
    for i in range(0,l):
      h = list(weather1['hourly'])[i] #each index of this will have each hour ka data

      one = {}
      temp = ['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM' , 'visibility' , 'pressure' , 'cloudcover' ,'DewPointF', 'WindGustKmph', 'tempF', 'WindChillF', 'humidity','time']

      for j in temp:
        one[j] = h[j]
      one['date'] = weather1['date']
      r = first['request'][0]
      one['airport'] = r['query']
      series = pd.Series(one) 
      df = df.append(series,True)
  return(df)
 

In [17]:
lis = os.listdir("drive/My Drive/weather/ATL")
n = []
li = list()
for i in lis:  #creating list of 2016-1,2,3.. and 2017-1,2,3 - n[], this is same for all airport
  if i.startswith("2016") or i.startswith("2017"):
    n.append(i)

weatherfiles = os.listdir("drive/My Drive/weather")
for b in weatherfiles:
  for i in n:
    f = open('drive/My Drive/weather/'+b+'/'+i) 
    data = json.load(f)
    li.append(weather_comp(data))
    f.close() 

df_weather = pd.concat(li,axis = 0)
print(df_weather)
#df.to_csv('data.csv')
#!cp data.csv "drive/My Drive/"


    windspeedKmph  ...                                            airport
0              15  ...  SFO, San Francisco International Airport, Unit...
1              14  ...  SFO, San Francisco International Airport, Unit...
2              12  ...  SFO, San Francisco International Airport, Unit...
3              11  ...  SFO, San Francisco International Airport, Unit...
4              11  ...  SFO, San Francisco International Airport, Unit...
..            ...  ...                                                ...
715             6  ...  DFW, Dallas/Fort Worth International Airport, ...
716             5  ...  DFW, Dallas/Fort Worth International Airport, ...
717             4  ...  DFW, Dallas/Fort Worth International Airport, ...
718             3  ...  DFW, Dallas/Fort Worth International Airport, ...
719             3  ...  DFW, Dallas/Fort Worth International Airport, ...

[263160 rows x 15 columns]


**Flight data Preprocessing**

In [18]:
year = ['2016','2017']
cols = ["FlightDate", "Quarter", "Year", "Month", "DayofMonth", "DepTime", "DepDel15" , "CRSDepTime" , "DepDelayMinutes" , "Origin", "Dest" , "ArrTime" , "CRSArrTime" , "ArrDel15" , "ArrDelayMinutes"]
df1 = pd.DataFrame(columns = cols)
req_air = ["ATL","CLT","DEN","DFW","EWR","IAH","JFK","LAS","LAX","MCO","MIA","ORD","PHX","SEA","SFO"]

for i in year:
  for j in range(1,13):
    f = "drive/My Drive/"+i+"/"+"On_Time_On_Time_Performance_"+i+"_"+str(j)+"/"+"On_Time_On_Time_Performance_"+i+"_"+str(j)+".csv"
    df = pd.read_csv(f)
    df1 = df1.append(df[df["Origin"].isin(req_air) & df["Dest"].isin(req_air)].loc[:,cols])


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (77) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (77,84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (48,77,84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (48,77,84,85) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local

In [19]:
df_flight = (pd.DataFrame.drop_duplicates(df1)).dropna()

**Merging**

In [20]:
df_weather['airport']=df_weather.airport.apply(lambda x: x[:3])
print(df_weather['airport'])

0      SFO
1      SFO
2      SFO
3      SFO
4      SFO
      ... 
715    DFW
716    DFW
717    DFW
718    DFW
719    DFW
Name: airport, Length: 263160, dtype: object


In [21]:
df_flight['CRSDepTime'] = df_flight.DepTime.apply(lambda x : str(int(int(x)/100)) if str(int(int(x)/100))!='24' else '0')
df_flight['CRSArrTime'] = df_flight.ArrTime.apply(lambda x : str(int(int(x)/100)) if str(int(int(x)/100))!='24' else '0')
print(df_flight['CRSArrTime'])

57        16
58        16
59        15
60        15
61        15
          ..
464194    13
464195    19
464197     9
464201    17
464203    12
Name: CRSArrTime, Length: 1851433, dtype: object


In [22]:
df_weather['time'] = df_weather['time'].apply(lambda x : str(int(int(x)/100)))
print(df_weather['time'])

0       0
1       1
2       2
3       3
4       4
       ..
715    19
716    20
717    21
718    22
719    23
Name: time, Length: 263160, dtype: object


In [23]:
df_pfinal = df_flight.merge(df_weather , left_on = ["CRSDepTime","FlightDate","Origin"], right_on = ['time','date','airport'] , how="left")
df_final = df_pfinal.merge(df_weather , left_on = ["CRSArrTime","FlightDate","Dest"], right_on = ['time','date','airport'] , how="left")
print(df_final)


         FlightDate Quarter  Year Month  ... humidity_y  time_y      date_y airport_y
0        2016-01-01       1  2016     1  ...         74      16  2016-01-01       JFK
1        2016-01-02       1  2016     1  ...         71      16  2016-01-02       JFK
2        2016-01-03       1  2016     1  ...         67      15  2016-01-03       JFK
3        2016-01-04       1  2016     1  ...         64      15  2016-01-04       JFK
4        2016-01-05       1  2016     1  ...         61      15  2016-01-05       JFK
...             ...     ...   ...   ...  ...        ...     ...         ...       ...
1851428  2017-12-17       4  2017    12  ...         77      13  2017-12-17       IAH
1851429  2017-12-17       4  2017    12  ...         83      19  2017-12-17       EWR
1851430  2017-12-17       4  2017    12  ...         87       9  2017-12-17       DEN
1851431  2017-12-17       4  2017    12  ...         58      17  2017-12-17       DEN
1851432  2017-12-17       4  2017    12  ...         7

In [24]:
df_final.to_csv('ProcessedData.csv')
!cp ProcessedData.csv "drive/My Drive/"

In [15]:
print(5+2)

7
